### Multiple turtlebot simulator with rapyuta.io.

#### 1.IDs
you can copy from rapyuta.io console

In [ ]:
AUTH_TOKEN = ''
PROJECT_ID = ''
DEPLOYMENT_PREFIX = 'yu_'
retry_count=90
interval=10

#gazebo
SIMULATOR_PACKAGE_ID = 'pkg-rbgbdcgakcfdptssqmpjomsz'
SIMULATOR_PLAN_ID = 'plan-hjqpauskkmcfujwxuyzeiloh'
SIMULATOR_COMPONENT = 'gazebo'

#rosbridge
ROSBRIDGE_PACKAGE_ID = 'pkg-lvxnevtdsusxgovjhaoxzcvr'
ROSBRIDGE_PLAN_ID = 'plan-warminnqaalumhkicczvebai'

#ui
UI_PACKAGE_ID = 'pkg-lybdwnlkqxyxnwosyjejjbjh'
UI_PLAN_ID = 'plan-nlihwyghctpjnucxuvswylcr'

#app_multi_tv1_external
APP_PACKAGE_ID = 'pkg-plfzsvvoauacihxxkepfrzvx'
APP_PLAN_ID = 'plan-aimtltlatcksawlbcitahcnf'
APP_COMPONENT = 'turtle'

#params
sim_params = {'WORLD':'turtlebot3_world', 'VNC_PASSWORD':'rapyuta'}
robot_params = [
    {'TURTLEBOT3_MODEL':'burger', 'ROBOT_NAME':'turtle0', 'X_POS':-0.5, 'Y_POS':-0.5, 'YAW':0.00},
    {'TURTLEBOT3_MODEL':'waffle', 'ROBOT_NAME':'turtle1', 'X_POS': 0.5, 'Y_POS': 0.5, 'YAW':3.14},
]
robot_num = len(robot_params)

#### 2. import module and create Client Instance

In [ ]:
from rapyuta_io import Client
from rapyuta_io.clients.package import ROSDistro

client = Client(AUTH_TOKEN, PROJECT_ID)

#### 3. Create routed network

In [ ]:
networks = client.get_all_routed_networks()
network = None
network_name = DEPLOYMENT_PREFIX + 'route_network'
for net in networks:
    if net.name == network_name and net.get_status().status == "Running":
        network = net
        print(DEPLOYMENT_PREFIX + 'route_network' + " was found and will be used")
        break
        
if not network:
    network = client.create_cloud_routed_network(network_name, ROSDistro.MELODIC, True)
    network.poll_routed_network_till_ready(retry_count=retry_count, sleep_interval=interval)
    print(DEPLOYMENT_PREFIX + 'route_network' + "was created successfully")

routed_networks = [network]

#### 3.Simulator deployment

In [ ]:
simulator_package = client.get_package(SIMULATOR_PACKAGE_ID)
simulator_configuration = simulator_package.get_provision_configuration(SIMULATOR_PLAN_ID)
for key in sim_params:
    simulator_configuration.add_parameter(SIMULATOR_COMPONENT, key, sim_params[key])
simulator_configuration.add_routed_networks(routed_networks)
simulator_deployment = simulator_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'sim',
				provision_configuration = simulator_configuration)
result = simulator_deployment.poll_deployment_till_ready(retry_count=50, sleep_interval=6)
print('gazebo started: ' + result['componentInfo'][0]['networkEndpoints']['vnc'])

#### 4.App deployments

In [ ]:
app_deployments = [None, None]
app_package = client.get_package(APP_PACKAGE_ID)
for i in range(robot_num):
    name = robot_params[i]['ROBOT_NAME']
    app_configuration = app_package.get_provision_configuration(APP_PLAN_ID)
    app_configuration.add_dependent_deployment(simulator_deployment)
    for key in robot_params[i]:
        app_configuration.add_parameter(APP_COMPONENT, key, robot_params[i][key])
    app_configuration.set_component_alias(APP_COMPONENT, name)
    app_configuration.add_routed_networks(routed_networks)
    app_deployments[i] = app_package.provision(
                    deployment_name = DEPLOYMENT_PREFIX + name,
                    provision_configuration = app_configuration)
    
for i in range(robot_num):
    app_deployments[i].poll_deployment_till_ready(retry_count=50, sleep_interval=6)
    print(DEPLOYMENT_PREFIX + robot_params[i]['ROBOT_NAME'] + ' started')

#### 5.Rosbridge deployment

In [ ]:
rosbridge_package = client.get_package(ROSBRIDGE_PACKAGE_ID)
rosbridge_configuration = rosbridge_package.get_provision_configuration(ROSBRIDGE_PLAN_ID)
# rosbridge_configuration.add_dependent_deployment(simulator_deployment)
for dep in app_deployments:
        rosbridge_configuration.add_dependent_deployment(dep)

rosbridge_configuration.add_routed_networks(routed_networks)
rosbridge_deployment = rosbridge_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'rosbridge' ,
				provision_configuration = rosbridge_configuration)
rosbridge_deployment.poll_deployment_till_ready(retry_count=50, sleep_interval=6)
print('rosbridge started')

#### 6.UI deployment

In [ ]:
ui_package = client.get_package(UI_PACKAGE_ID)
ui_configuration = ui_package.get_provision_configuration(UI_PLAN_ID)
ui_configuration.add_dependent_deployment(rosbridge_deployment)
        
ui_deployment = ui_package.provision(
                deployment_name = DEPLOYMENT_PREFIX + 'ui',
				provision_configuration = ui_configuration)
result = ui_deployment.poll_deployment_till_ready()
print('ui started ' + result['componentInfo'][0]['networkEndpoints']['UserInterface'])

#### 7.Deplovision all deploment

In [ ]:
retry_limit = 3
simulator_deployment.deprovision(retry_limit)
for i in range(robot_num):
    app_deployments[i].deprovision(retry_limit)
rosbridge_deployment.deprovision(retry_limit)
ui_deployment.deprovision(retry_limit)